In [13]:
import os
import numpy as np

## 5.19 Rotate a 2D Array 

Image rotation is a fundamental operation in computer graphics. Figure 5.4 illustrates the rotation operation on a 2D array representing a bit-map of an image. Specifically, the image is rotated by 90 degrees clockwise.

Write a function that takes as input an n x n 2D array, and rotates the array by 90 degrees clockwise.
Hint: Focus on the boundary elements.

In [69]:
def rotate_90_clock_np(arr):
    arr = np.array(arr)
    arr_rot = np.zeros(arr.shape)
    for old_row_ind, new_col_ind in enumerate(range(len(arr))[::-1]):
        arr_rot[:, new_col_ind] = arr[old_row_ind, :]
    return arr_rot


def rotate_coords_90_clock(i, j, n):
    i_rot = j
    j_rot = n - 1 - i
    return i_rot, j_rot


def rotate_corners(arr, i_start, j_start):
    n = len(arr)
    i_orig, j_orig = i_start, j_start
    reassign_val = arr[i_orig][j_orig]
    for _ in range(4):
        i_rot, j_rot = rotate_coords_90_clock(i_orig, j_orig, n)
        temp = arr[i_rot][j_rot]
        arr[i_rot][j_rot] = reassign_val
        reassign_val = temp
        i_orig, j_orig = i_rot, j_rot

        
def rotate_square(arr, ind_start):
    i_start = ind_start
    for j_start in range(ind_start, len(arr) - ind_start - 1):
        rotate_corners(arr, i_start, j_start)

        
def rotate_90_clock_inplace(arr):
    num_square = len(arr) / 2
    for ind_start in range(num_square):
        rotate_square(arr, ind_start)

        
def rotate_90_clock(arr):
    n = len(arr)
    arr_rot = [[None] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            i_rot, j_rot = rotate_coords_90_clock(i, j, n)
            arr_rot[i_rot][j_rot] = arr[i][j]
    return arr_rot


def rotate(arr):
    for i in xrange(len(arr)/2):
#         for j in xrange(len(arr)/2 + len(arr)%2):
        for j in xrange(i, len(arr) - i - 1):
            rotate_corners(arr,i,j)
    return arr

In [70]:
# arr = [[i] * n for i in range(5)]
# arr = [[1, 2], [3, 4]]
# arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
arr = [[1, 2, 3, 4, 140], [5, 6, 7, 8, 130], [9, 10, 11, 12, 120], [13, 14, 15, 16, 110], [17, 18, 19, 20, 100]]

# arr_rot = rotate_90_clock(arr)

# print(np.array(arr))
# print('')
# print(np.array(arr_rot))

print(np.array(arr))
# rotate_90_clock_inplace(arr)
rotate(arr)
print('')
print(np.array(arr))

[[  1   2   3   4 140]
 [  5   6   7   8 130]
 [  9  10  11  12 120]
 [ 13  14  15  16 110]
 [ 17  18  19  20 100]]

[[ 17  13   9   5   1]
 [ 18  14  10   6   2]
 [ 19  15  11   7   3]
 [ 20  16  12   8   4]
 [100 110 120 130 140]]


## 5.13 Sample Online Data 

This problem is motivated by the disgn of a packet sniffer that provides a uniform sample of packets for a network session.

Design a program that takes as input a size k, and reads packets, coninuously maintaining a uniform random subset of size k of the read packets.

Hint: Suppose you have a procedure which selects k packets from the first n >= k packets as specified. How would you deal with the (n+1)st packet?

In [2]:
import random

class PacketSampler:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = float(self.k) / (self.n + 1)
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


class PacketSamplerWrong:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = random.random()
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


def get_packet_subset(n, packet_sampler):
    for i in range(n):
        packet_sampler.receive_packet(i)
    return packet_sampler.subset


def get_simulated_distribution(n, k, sampler_name, num_trials):
    if sampler_name == 'right':
        Sampler = PacketSampler
    elif sampler_name == 'wrong':
        Sampler = PacketSamplerWrong
    else:
        raise ValueError('Unrecognized sampler name.')
    
    packet_counter = {p: 0 for p in range(n)}
    for trial_ind in range(num_trials):
        subset = get_packet_subset(n, Sampler(k))
        for p in subset:
            packet_counter[p] += 1
    
    return packet_counter

In [7]:
num_trials = 500000
n = 10
k = 3
sampler_name = 'wrong'

packet_counts = get_simulated_distribution(n=n, k=k, sampler_name=sampler_name, num_trials=num_trials)
num_exp = num_trials / n * k

for count in packet_counts.itervalues():
    pct_dev = 100.0 * abs(num_exp - count) / num_exp
    print(pct_dev)

7.096
6.99266666667
6.67866666667
44.226
33.2373333333
19.6773333333
3.67266666667
15.8506666667
38.8246666667
66.9053333333


## 5.18 Compute the Spiral Ordering of a 2D array 

In [48]:
def change_direction(increment):
    increment_new = increment[::-1]
    if abs(increment[0]) == 1:
        increment_new[1] *= -1
    return increment_new

def spiralize(arr):
    i_bounds = [0, len(arr) - 1]
    j_bounds = [0, len(arr[0]) - 1]
    increment = [0, 1]
    i = 0
    j = 0
    arr_spiral = []

    while i_bounds[1] >= i_bounds[0]:

        arr_spiral.append(arr[i][j])

        i_prop = i + increment[0]
        j_prop = j + increment[1]

        if j_prop > j_bounds[1]:
            i_bounds[0] += 1
        elif j_prop < j_bounds[0]:
            i_bounds[1] -= 1
        elif i_prop > i_bounds[1]:
            j_bounds[1] -= 1
        elif i_prop < i_bounds[0]:
            j_bounds[0] += 1
        else:
            i = i_prop
            j = j_prop
            continue

        increment = change_direction(increment)
        i += increment[0]
        j += increment[1]
        
    return arr_spiral

In [49]:
arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

print(np.array(arr))
print('')
print(spiralize(arr))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

[1, 2, 3, 4, 8, 12, 16, 15, 14, 13, 9, 5, 6, 7, 11, 10]


## 6.13: Find the first occurrence of a substring 

A good string search algorithm is fundamental to the performance of many applications. Several clever algorithms have been proposed for string search, each with its own trade-offs. As a result, there is no single perfect answer. If someone asks you this question in an interview, the best way to approach this problem would be to work through one good algorithm in detail and discuss at a high level other algorithms.

Given two strings s (the "search string") and t (the "text"), find the first occurrence of s in t.

Hint: Form a signature from a string

In [81]:
main_string = 'gshgiuregbisnreipngseproingseonvdk;nfvdklldksfl'
substring = 'gbisn'



def find_substring_start_ind_simple(main_string, substring):
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        num_match = 0
        for i_sub, i_main in enumerate(range(start_ind, start_ind + substring_len)):
            num_match += substring[i_sub] == main_string[i_main]
        if num_match == substring_len:
            return start_ind
    return None


def find_substring_start_ind_hash(main_string, substring):
    substring_hash = set([substring])
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if main_string[start_ind:start_ind + substring_len] in substring_hash:
            return start_ind
    return None


class RollingHasher:
    
    def __init__(self, first_substring, base=256, prime_modulus=101):
        self.first_substring = first_substring
        self.base = base
        self.prime_modulus = prime_modulus
        self._val_to_remove = None
        self._last_hash = None
        
    def get_hash(next_letter=None):
        if next_letter is None:
            hash_sum = 0
            for c in self.first_substring:
                
                
            
        
        
    
        


print(find_substring_start_ind_simple(main_string, substring))
print(find_substring_start_ind_hash(main_string, substring))

8
8


In [2]:
ord('a')

97